In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import seaborn as sns
import scipy.stats as stats
from scipy.stats import anderson
import csv
# from pathos.multiprocessing import ThreadPool as Pool #多线程

from joblib import Parallel, delayed

In [17]:
data = pd.read_csv("C:/Users/Admin/数据处理代码/user7_combined_output.csv")

data = np.array(data['velocity'])

In [18]:
df = np.array(data)
print(df[0])

2.189256509876819


In [19]:
def ApEn2(s, r, m, threads):
    s = np.squeeze(s)
    th = r * np.std(s) #容限阈值
    
    def phi(m):
        n = len(s)
        x = s[np.arange(n-m+1).reshape(-1,1) + np.arange(m)]
        print('phi完成')
        def ci(xi):
            return ((np.abs(x-xi).max(1) <= th).sum()) / (n-m+1)
        
        c = threads.map(ci, x)
        print('ci完成')
        return np.sum(np.log(c)) / (n-m+1)
    
    return phi(m) - phi(m+1)

In [20]:
def approximate_entropy(x, m=2, r=0.2):
    """
    计算近似熵
    x: 时间序列
    m: 子序列的长度，默认为2
    r: 差值系数，默认为0.2
    """
    
    n = len(x)
    # 计算单个子序列的欧几里得距离
    def distance(xi, xj):
        return np.abs(xi - xj)

    # 计算两个子序列之间的最大差值
    def max_distance(subsequence, subsequence_i):
        return np.max(np.abs(subsequence - subsequence_i))

    a = 0
    b = 0
    for i in range(n-m-1):
        a += np.sum(distance(x[i:i+m], x[i+m]) <= r)
        b += np.sum(distance(x[i:i+m+1], x[i+m+1]) <= r)
    return -np.log(a/b)

In [21]:
def calculate_entropy(i, df, r):
   return approximate_entropy(df, i, r)

In [25]:
entropy_list = []
std = np.std(df)
r = std * 0.2

low_bound = 10
up_bound = 300

# Parallel computation of entropies
entropy_list = Parallel(n_jobs=-1)(
   delayed(calculate_entropy)(i, df, r) 
   for i in range(low_bound, up_bound)
)

print("Completed calculating entropies")


# Create array with lengths and entropies
lengths = np.arange(low_bound, up_bound)
data = pd.DataFrame({
   'length': range(low_bound, up_bound),
   'entropy': entropy_list
})
data.to_csv('entropy_results2.csv', index=False)

Completed calculating entropies
